In [188]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [189]:
teste = pd.read_csv('teste.csv')

In [190]:
label_resposta = LabelEncoder()
teste['reposta'] = label_resposta.fit_transform(teste['reposta'].astype(str))
teste.head(10)

,Pergunta 1,Pergunta 2,Pergunta 3,Pergunta 4,Pergunta 5,Pergunta 6,Pergunta 7,Pergunta 8,Pergunta 9,Pergunta 10,Pergunta 11,Pergunta 12,reposta
0,1,1,1,1,0,1,0,0,1,0,1,1,0
1,1,1,1,0,0,0,1,0,1,1,0,0,0
2,1,0,1,1,0,1,0,1,1,0,1,0,0
3,0,1,0,1,1,1,1,1,1,0,1,1,1
4,1,0,1,0,1,1,0,1,0,0,0,0,1
5,0,0,0,1,0,1,1,1,1,0,1,1,1
6,1,0,1,1,0,0,0,0,1,1,1,1,2
7,0,1,1,0,0,1,0,0,1,1,1,0,2
8,1,0,0,1,0,1,1,0,1,1,1,0,2
9,1,1,1,1,1,1,1,1,1,1,1,1,3


In [191]:
X = teste.drop('reposta', axis=1)
y = teste['reposta']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [192]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_train[:10]

array([[-1.29099445, -1.        , -1.73205081,  0.57735027, -0.77459667,
         0.37796447,  1.29099445,  0.77459667,  0.37796447, -0.77459667,
         0.37796447,  0.77459667],
       [ 0.77459667,  1.        ,  0.57735027,  0.57735027, -0.77459667,
         0.37796447, -0.77459667, -1.29099445,  0.37796447, -0.77459667,
         0.37796447,  0.77459667],
       [-1.29099445,  1.        ,  0.57735027, -1.73205081, -0.77459667,
         0.37796447, -0.77459667, -1.29099445,  0.37796447,  1.29099445,
         0.37796447, -1.29099445],
       [ 0.77459667, -1.        ,  0.57735027,  0.57735027, -0.77459667,
         0.37796447, -0.77459667,  0.77459667,  0.37796447, -0.77459667,
         0.37796447, -1.29099445],
       [ 0.77459667,  1.        ,  0.57735027,  0.57735027,  1.29099445,
         0.37796447,  1.29099445,  0.77459667,  0.37796447,  1.29099445,
         0.37796447,  0.77459667],
       [ 0.77459667, -1.        ,  0.57735027, -1.73205081,  1.29099445,
         0.37796447, -

In [193]:
estimator = DecisionTreeClassifier(max_leaf_nodes=6, random_state=2)
estimator.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=6, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=2, splitter='best')

In [194]:
n_nodes = estimator.tree_.node_count
print(n_nodes)
children_left = estimator.tree_.children_left
children_right = estimator.tree_.children_right
feature = estimator.tree_.feature
threshold = estimator.tree_.threshold

9


In [195]:
node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)

In [196]:
stack = [(0, -1)]
while len(stack) > 0:
    node_id, parent_depth = stack.pop()
    node_depth[node_id] = parent_depth + 1

    # If we have a test node
    if (children_left[node_id] != children_right[node_id]):
        stack.append((children_left[node_id], parent_depth + 1))
        stack.append((children_right[node_id], parent_depth + 1))
    else:
        is_leaves[node_id] = True

In [197]:
print("The binary tree structure has %s nodes and has "
      "the following tree structure:"
      % n_nodes)
for i in range(n_nodes):
    if is_leaves[i]:
        print("%snode=%s leaf node." % (node_depth[i] * "\t", i))
    else:
        print("%snode=%s test node: go to node %s if X[:, %s] <= %s else to "
              "node %s."
              % (node_depth[i] * "\t",
                 i,
                 children_left[i],
                 feature[i],
                 threshold[i],
                 children_right[i],
                 ))
print()

The binary tree structure has 9 nodes and has the following tree structure:
node=0 test node: go to node 1 if X[:, 9] <= 0.2581988573074341 else to node 2.
	node=1 test node: go to node 5 if X[:, 6] <= 0.2581988573074341 else to node 6.
	node=2 test node: go to node 3 if X[:, 7] <= -0.2581988573074341 else to node 4.
		node=3 leaf node.
		node=4 leaf node.
		node=5 test node: go to node 7 if X[:, 4] <= 0.2581988573074341 else to node 8.
		node=6 leaf node.
			node=7 leaf node.
			node=8 leaf node.



In [198]:
node_indicator = estimator.decision_path(X_test)
leave_id = estimator.apply(X_test)
sample_id = 0
node_index = node_indicator.indices[node_indicator.indptr[sample_id]:
                                    node_indicator.indptr[sample_id + 1]]

print('Rules used to predict sample %s: ' % sample_id)

Rules used to predict sample 0: 


In [199]:
for node_id in node_index:
    if leave_id[sample_id] != node_id:
        continue

    if (X_test[sample_id, feature[node_id]] <= threshold[node_id]):
        threshold_sign = "<="
    else:
        threshold_sign = ">"

    print("decision id node %s : (X_test[%s, %s] (= %s) %s %s)"
          % (node_id,
             sample_id,
             feature[node_id],
             X_test[sample_id, feature[node_id]],
             threshold_sign,
             threshold[node_id]))


decision id node 3 : (X_test[0, -2] (= 0.3779644730092272) > -2.0)


In [200]:
sample_ids = [0, 1]
common_nodes = (node_indicator.toarray()[sample_ids].sum(axis=0) ==
                len(sample_ids))

common_node_id = np.arange(n_nodes)[common_nodes]

print("\nThe following samples %s share the node %s in the tree"
      % (sample_ids, common_node_id))
print("It is %s %% of all nodes." % (100 * len(common_node_id) / n_nodes,))


The following samples [0, 1] share the node [0 2 3] in the tree
It is 33.333333333333336 % of all nodes.
